<a href="https://colab.research.google.com/github/OumaymaRedissi/Simulation/blob/main/edit_chargers_lxml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Editing charger stations file to match matsim format

In [1]:
!pip install lxml
import pandas as pd 
#help(etree)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import xml.etree.ElementTree as ET
# parsing directly.
tree = ET.parse('charging_stations.xml')
root = tree.getroot()
# printing the root.
print(root)

<Element 'osm' at 0x7f75c5a33230>


In [3]:
def is_float(element) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

**pp_formatter()** to remove the white spaces and deletes everything after the 'kw'

In [4]:
def pp_formatter(plug_power:str) -> str:
    start_index = plug_power.find("kW")
    plug_power = (plug_power[0: start_index:] + plug_power[len(plug_power) + 1::]).replace(' ','')
    return plug_power
    

In [5]:
def extractor() :
  df = pd.DataFrame()
  for node in root.findall('node'):
      id = node.get('id')
      lat = node.get('lat')
      lon = node.get('lon')
      capacity=2 #default capacity =2
      list_of_plug_powers=[]
      for tag in node.findall('tag'):
        att_k=tag.get('k')
        att_v=tag.get('v')
        if att_k == "capacity":
          if is_float(att_v):
            capacity = int(att_v)
        if "kW" in att_v :
          plug_power_str = pp_formatter(att_v)
          if is_float(plug_power_str):
            list_of_plug_powers.append(float(plug_power_str))
      plug_power = max(list_of_plug_powers, default=22)#default plug power=22
      #print(id,lat,lon,plug_power,capacity)
      data = {'id':id,'x':lat,'y':lon,'plug_power':plug_power,'plug_count':capacity}
      df = df.append(data,ignore_index=True)
  
  #print(df)
  return df
       

print(extractor())

              id           x          y  plug_power  plug_count
0     1530997538  48.8822680  2.3707230        22.0         2.0
1     1601711126  48.9157408  2.2909357        22.0         2.0
2     1601711127  48.9157762  2.2909034        22.0         2.0
3     1601711128  48.9158113  2.2908713        22.0         2.0
4     1601711129  48.9158491  2.2908367        22.0         2.0
...          ...         ...        ...         ...         ...
1450  9791175803  48.8555664  2.3150384        22.0         6.0
1451  9792084981  49.1797976  3.1234451        22.0         2.0
1452  9797611182  48.8994232  2.3710185        22.0         6.0
1453  9807217413  48.8541853  2.3452424        22.0         5.0
1454  9811956662  48.7095969  2.1710766        22.0         2.0

[1455 rows x 5 columns]


In [6]:
df = extractor()
df.to_csv('charging_station.csv')

In [30]:
root= ET.Element("root")
chargers = ET.SubElement(root, "chargers")

ET.tostring(root)

for row in df.itertuples() : 
  charger = ET.SubElement(chargers, "charger", id=str(row.id), x=str(row.x), y=str(row.y), plug_power=str(row.plug_power), plug_count=str(row.plug_count))
ET.dump(root);
ET.parse('charging_stations.xml');



<root><chargers><charger id="1530997538" plug_count="2.0" plug_power="22.0" x="48.8822680" y="2.3707230" /><charger id="1601711126" plug_count="2.0" plug_power="22.0" x="48.9157408" y="2.2909357" /><charger id="1601711127" plug_count="2.0" plug_power="22.0" x="48.9157762" y="2.2909034" /><charger id="1601711128" plug_count="2.0" plug_power="22.0" x="48.9158113" y="2.2908713" /><charger id="1601711129" plug_count="2.0" plug_power="22.0" x="48.9158491" y="2.2908367" /><charger id="1601711130" plug_count="2.0" plug_power="22.0" x="48.9158862" y="2.2908028" /><charger id="1601711132" plug_count="2.0" plug_power="22.0" x="48.9159218" y="2.2907703" /><charger id="1685216879" plug_count="2.0" plug_power="7.4" x="48.7942262" y="2.3344777" /><charger id="1714033524" plug_count="2.0" plug_power="7.4" x="48.7928962" y="2.3216763" /><charger id="1756259321" plug_count="4.0" plug_power="22.0" x="48.8337990" y="2.3085854" /><charger id="2003145869" plug_count="4.0" plug_power="22.0" x="48.8597459" y